# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 05:11:28] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=34813, nccl_port=None, mem_fraction_static=0.849, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=850320450, constrained_json_w

[2025-07-09 05:11:28] Inferred chat template from model path: llama-2


[2025-07-09 05:11:36] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 05:11:36] Init torch distributed begin.


[2025-07-09 05:11:36] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 05:11:37] Load weight begin. avail mem=31.64 GB


...

[2025-07-09 05:11:42] Init torch distributed begin.
[2025-07-09 05:11:42] Init torch distributed ends. mem usage=0.00 GB
[2025-07-09 05:11:42] Load weight begin. avail mem=6.84 GB
[2025-07-09 05:11:42] The weight of LmHead is not packed
[2025-07-09 05:11:42] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.35s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.35s/it]

[2025-07-09 05:11:44] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=5.81 GB, mem usage=1.03 GB.
[2025-07-09 05:11:44] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-07-09 05:11:44] Memory pool end. avail mem=5.49 GB


[2025-07-09 05:11:44] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=5.72 GB


[2025-07-09 05:11:44] INFO:     Started server process [2948751]
[2025-07-09 05:11:44] INFO:     Waiting for application startup.
[2025-07-09 05:11:44] INFO:     Application startup complete.
[2025-07-09 05:11:44] INFO:     Uvicorn running on http://127.0.0.1:34813 (Press CTRL+C to quit)
[2025-07-09 05:11:44] INFO:     127.0.0.1:56074 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 05:11:45] INFO:     127.0.0.1:56076 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 05:11:45] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:11:45.828840


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-09 05:12:52] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, #token: 7, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-09T05:12:52.089728


[2025-07-09 05:13:23] INFO:     127.0.0.1:56090 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 05:13:23] The server is fired up and ready to roll!


[2025-07-09 05:13:26] INFO:     127.0.0.1:56102 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-07-09 05:13:26] Child process unexpectedly failed with exitcode=9. pid=2949323


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 05:13:34] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=33446, nccl_port=None, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=208104655, constrained_json_whi

[2025-07-09 05:13:35] Inferred chat template from model path: llama-2


[2025-07-09 05:13:42] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 05:13:42] Init torch distributed begin.


[2025-07-09 05:13:43] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 05:13:44] Load weight begin. avail mem=43.69 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.28s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.28s/it]

[2025-07-09 05:13:50] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=54.23 GB, mem usage=-6.75 GB.
[2025-07-09 05:13:50] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-07-09 05:13:50] Memory pool end. avail mem=53.92 GB


[2025-07-09 05:13:50] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=54.10 GB


[2025-07-09 05:13:50] INFO:     Started server process [2956664]
[2025-07-09 05:13:50] INFO:     Waiting for application startup.
[2025-07-09 05:13:50] INFO:     Application startup complete.
[2025-07-09 05:13:50] INFO:     Uvicorn running on http://127.0.0.1:33446 (Press CTRL+C to quit)


[2025-07-09 05:13:51] INFO:     127.0.0.1:59706 - "GET /v1/models HTTP/1.1" 200 OK
[2025-07-09 05:13:51] INFO:     127.0.0.1:59722 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 05:13:51] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:13:51.872217


[2025-07-09 05:13:54] INFO:     127.0.0.1:59728 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 05:13:54] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-09 05:13:56] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:13:56.781961


[2025-07-09 05:13:57] INFO:     127.0.0.1:33948 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-07-09 05:13:57] Child process unexpectedly failed with exitcode=9. pid=2956888
[2025-07-09 05:13:57] Child process unexpectedly failed with exitcode=9. pid=2956816


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 05:14:04] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=38667, nccl_port=None, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=487314391,

[2025-07-09 05:14:13] Casting torch.bfloat16 to torch.float16.


[2025-07-09 05:14:13] Casting torch.bfloat16 to torch.float16.
[2025-07-09 05:14:13] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 05:14:13] Init torch distributed begin.


[2025-07-09 05:14:14] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 05:14:15] Load weight begin. avail mem=75.92 GB


...

[2025-07-09 05:14:34] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=42.60 GB


[2025-07-09 05:14:35] INFO:     Started server process [2958733]
[2025-07-09 05:14:35] INFO:     Waiting for application startup.
[2025-07-09 05:14:35] INFO:     Application startup complete.
[2025-07-09 05:14:35] INFO:     Uvicorn running on http://127.0.0.1:38667 (Press CTRL+C to quit)


[2025-07-09 05:14:36] INFO:     127.0.0.1:47756 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 05:14:36] INFO:     127.0.0.1:47772 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 05:14:36] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:14:36.678609


[2025-07-09 05:14:39] INFO:     127.0.0.1:47780 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 05:14:39] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-09 05:14:41] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:14:41.293244


[2025-07-09 05:14:41] INFO:     127.0.0.1:47786 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-07-09 05:14:41] Child process unexpectedly failed with exitcode=9. pid=2959194
[2025-07-09 05:14:41] Child process unexpectedly failed with exitcode=9. pid=2959061


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 05:14:49] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=33607, nccl_port=None, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=35747680, constrain

[2025-07-09 05:14:57] Casting torch.bfloat16 to torch.float16.


[2025-07-09 05:14:58] Casting torch.bfloat16 to torch.float16.
[2025-07-09 05:14:58] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 05:14:58] Init torch distributed begin.


[2025-07-09 05:14:58] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 05:15:00] Load weight begin. avail mem=62.20 GB


...

[2025-07-09 05:15:17] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=58.56 GB


[2025-07-09 05:15:18] INFO:     Started server process [2960959]
[2025-07-09 05:15:18] INFO:     Waiting for application startup.
[2025-07-09 05:15:18] INFO:     Application startup complete.
[2025-07-09 05:15:18] INFO:     Uvicorn running on http://127.0.0.1:33607 (Press CTRL+C to quit)


[2025-07-09 05:15:18] INFO:     127.0.0.1:43776 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 05:15:19] INFO:     127.0.0.1:43782 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 05:15:19] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:15:19.448006


[2025-07-09 05:15:22] INFO:     127.0.0.1:43784 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 05:15:22] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-09 05:15:23] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:15:23.872034


[2025-07-09 05:15:24] INFO:     127.0.0.1:43798 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-07-09 05:15:24] Child process unexpectedly failed with exitcode=9. pid=2961527
[2025-07-09 05:15:24] Child process unexpectedly failed with exitcode=9. pid=2961394


## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 05:15:32] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='XiaomiMiMo/MiMo-7B-RL', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=31410, nccl_port=None, mem_fraction_static=0.5, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=204159420, constrained_json_whitespace_pattern=None, watch

[2025-07-09 05:15:44] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 05:15:44] Init torch distributed begin.
[2025-07-09 05:15:44] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 05:15:46] Load weight begin. avail mem=78.50 GB


[2025-07-09 05:15:47] The weight of LmHead is not packed
[2025-07-09 05:15:47] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-07-09 05:15:53] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=56.97 GB


[2025-07-09 05:15:53] INFO:     Started server process [2962838]
[2025-07-09 05:15:53] INFO:     Waiting for application startup.
[2025-07-09 05:15:53] INFO:     Application startup complete.
[2025-07-09 05:15:53] INFO:     Uvicorn running on http://0.0.0.0:31410 (Press CTRL+C to quit)


[2025-07-09 05:15:54] INFO:     127.0.0.1:44478 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 05:15:54] INFO:     127.0.0.1:44492 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 05:15:54] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:15:54.846148


[2025-07-09 05:15:55] INFO:     127.0.0.1:44496 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 05:15:55] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-07-09 05:15:59] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:15:59.265073


[2025-07-09 05:15:59] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, accept len: 1.79, cuda graph: False, gen throughput (token/s): 10.80, #queue-req: 0, timestamp: 2025-07-09T05:15:59.866208


[2025-07-09 05:16:00] Decode batch. #running-req: 1, #token: 162, token usage: 0.01, accept len: 1.75, cuda graph: False, gen throughput (token/s): 109.87, #queue-req: 0, timestamp: 2025-07-09T05:16:00.503346


[2025-07-09 05:16:01] Decode batch. #running-req: 1, #token: 228, token usage: 0.01, accept len: 1.65, cuda graph: False, gen throughput (token/s): 102.32, #queue-req: 0, timestamp: 2025-07-09T05:16:01.148372


[2025-07-09 05:16:01] Decode batch. #running-req: 1, #token: 298, token usage: 0.01, accept len: 1.75, cuda graph: False, gen throughput (token/s): 107.77, #queue-req: 0, timestamp: 2025-07-09T05:16:01.797901


[2025-07-09 05:16:02] Decode batch. #running-req: 1, #token: 361, token usage: 0.02, accept len: 1.57, cuda graph: False, gen throughput (token/s): 98.12, #queue-req: 0, timestamp: 2025-07-09T05:16:02.439964


[2025-07-09 05:16:03] Decode batch. #running-req: 1, #token: 434, token usage: 0.02, accept len: 1.82, cuda graph: False, gen throughput (token/s): 114.40, #queue-req: 0, timestamp: 2025-07-09T05:16:03.078087
[2025-07-09 05:16:03] INFO:     127.0.0.1:44500 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

[2025-07-09 05:16:03] Child process unexpectedly failed with exitcode=9. pid=2963054


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).